## Dependencies

In [1]:
import gc
import re
import warnings
from joblib import load
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation, Average, Add, Reshape
from tensorflow.keras.layers import GlobalAveragePooling1D, Embedding, LSTM, Conv1D, SpatialDropout1D, Bidirectional, Flatten
from googleqa_utilityscript import *


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
module_url = '/kaggle/input/universalsentenceencodermodels/universal-sentence-encoder-models/use'
encoder_path = '/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'
model_path = '/kaggle/input/84-googleq-a-train-glove-lstm-10paragraphs-v2/model.h5'
tokenizer_path = '/kaggle/input/84-googleq-a-train-glove-lstm-10paragraphs-v2/tokenizer.joblib'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

for feature in text_features:
    test[feature + '_uncased'] = test[feature]

In [5]:
def split_paragraphs(row):
    splited_body = re.split('\n\n\n\n\n|\n\n\n\n|\n\n\n|\n\n', row['question_body_uncased'])
    splited_answer = re.split('\n\n\n\n\n|\n\n\n\n|\n\n\n|\n\n', row['answer_uncased'])
    for i in range(len(splited_body))[:10]:
        row['body_paragraph_%s' % (i+1)] = splited_body[i]
    for i in range(len(splited_answer))[:10]:
        row['answer_paragraph_%s' % (i+1)] = splited_answer[i]
    return row

test = test.apply(split_paragraphs, axis=1)

# Model parameters

In [6]:
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
MAX_FEATURES = 100000
TITLE_MAX_LEN = 30
BODY_MAX_LEN = 30
ANSWER_MAX_LEN = 30

## Test set

In [7]:
body_ft = ['body_paragraph_1', 'body_paragraph_2', 'body_paragraph_3', 'body_paragraph_4', 'body_paragraph_5', 
           'body_paragraph_6', 'body_paragraph_7', 'body_paragraph_8', 'body_paragraph_9', 'body_paragraph_10']
answer_ft = ['answer_paragraph_1', 'answer_paragraph_2', 'answer_paragraph_3', 'answer_paragraph_4', 'answer_paragraph_5', 
             'answer_paragraph_6', 'answer_paragraph_7', 'answer_paragraph_8', 'answer_paragraph_9', 'answer_paragraph_10']

tokenizer = load(tokenizer_path)


# Test features
# X_test_title = test['question_title']
# X_test_body = test['question_body']
# X_test_answer = test['answer']
X_test_title_seq = test['question_title_uncased']
X_test_body_fts = test[body_ft].astype(str)
X_test_answer_fts = test[answer_ft].astype(str)

# Tokenize the sentences
X_test_title_seq = tokenizer.texts_to_sequences(X_test_title_seq)
for ft in body_ft:
    X_test_body_fts[ft] = tokenizer.texts_to_sequences(X_test_body_fts[ft])
for ft in answer_ft:
    X_test_answer_fts[ft] = tokenizer.texts_to_sequences(X_test_answer_fts[ft])

X_test_body_seq = np.empty((X_test_body_fts.shape[0], BODY_MAX_LEN, X_test_body_fts.shape[1]))
X_test_answer_seq = np.empty((X_test_answer_fts.shape[0], ANSWER_MAX_LEN, X_test_answer_fts.shape[1]))

# Pad the sentences
X_test_title_seq = pad_sequences(X_test_title_seq, maxlen=TITLE_MAX_LEN)
for index, ft in enumerate(body_ft):
    X_test_body_seq[...,index] = pad_sequences(X_test_body_fts[ft], maxlen=BODY_MAX_LEN)
for index, ft in enumerate(answer_ft):
    X_test_answer_seq[...,index] = pad_sequences(X_test_answer_fts[ft], maxlen=ANSWER_MAX_LEN)
    
X_test = [X_test_title_seq, X_test_body_seq, X_test_answer_seq]

# Model

In [8]:
# use_embed = hub.load(module_url)

# def USEEmbedding(x):
#     return use_embed(tf.squeeze(tf.cast(x, tf.string)))

# def encoder_block(input_layer):
#     encoder = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_layer)
    
#     return encoder

def encoder_block_parag(input_layer, index, name_pre):
    name = '%s_parag_%d' % (name_pre, index)
    encoder = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False, name='enc_'+name)(input_layer[...,index])
    encoder = SpatialDropout1D(0.2)(encoder)
    encoder = LSTM(64, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=False)(encoder)

    return encoder

## Load embedding

In [9]:
glove_embedding_matrix, glove_unknown_words = build_matrix(tokenizer.word_index, encoder_path, MAX_FEATURES)
embedding_name = 'GloVe'
print('n unknown words "%s": %d' % (embedding_name, len(glove_unknown_words)))
print('"%s" embedding shape: %s' % (embedding_name, glove_embedding_matrix.shape))

n unknown words "GloVe": 19913
"GloVe" embedding shape: (58007, 300)


In [10]:
# Title sequence
input_title_seq = Input(shape=(TITLE_MAX_LEN,), dtype=tf.float32, name='input_title_seq')
title = Embedding(*glove_embedding_matrix.shape, weights=[glove_embedding_matrix], trainable=False)(input_title_seq)
title = SpatialDropout1D(0.2)(title)
title = LSTM(128, kernel_initializer='lecun_normal', recurrent_dropout=0.5, return_sequences=True)(title)
title_out = GlobalAveragePooling1D()(title)

# Body sequence
input_body_seq = Input(shape=(*X_test_body_seq.shape[1:], ), dtype=tf.float32, name='input_body_seq')

body1_out = encoder_block_parag(input_body_seq, 0, 'body')
body2_out = encoder_block_parag(input_body_seq, 1, 'body')
body3_out = encoder_block_parag(input_body_seq, 2, 'body')
body4_out = encoder_block_parag(input_body_seq, 3, 'body')
body5_out = encoder_block_parag(input_body_seq, 4, 'body')
body6_out = encoder_block_parag(input_body_seq, 5, 'body')
body7_out = encoder_block_parag(input_body_seq, 6, 'body')
body8_out = encoder_block_parag(input_body_seq, 7, 'body')
body9_out = encoder_block_parag(input_body_seq, 8, 'body')
body10_out = encoder_block_parag(input_body_seq, 9, 'body')


# Answer sequence
input_answer_seq = Input(shape=(*X_test_body_seq.shape[1:], ), dtype=tf.float32, name='input_answer_seq')

answer1_out = encoder_block_parag(input_answer_seq, 0, 'answer')
answer2_out = encoder_block_parag(input_answer_seq, 1, 'answer')
answer3_out = encoder_block_parag(input_answer_seq, 2, 'answer')
answer4_out = encoder_block_parag(input_answer_seq, 3, 'answer')
answer5_out = encoder_block_parag(input_answer_seq, 4, 'answer')
answer6_out = encoder_block_parag(input_answer_seq, 5, 'answer')
answer7_out = encoder_block_parag(input_answer_seq, 6, 'answer')
answer8_out = encoder_block_parag(input_answer_seq, 7, 'answer')
answer9_out = encoder_block_parag(input_answer_seq, 8, 'answer')
answer10_out = encoder_block_parag(input_answer_seq, 9, 'answer')


# Output
x = Concatenate()([title_out, body1_out, body2_out, body3_out, body4_out, body5_out, body6_out, body7_out, body8_out, body9_out, body10_out, answer1_out, answer2_out, answer3_out, answer4_out, answer5_out, answer6_out, answer7_out, answer8_out, answer9_out, answer10_out])
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', kernel_initializer='lecun_normal')(x)
x = Dropout(0.5)(x)

output = Dense(N_CLASS, activation='sigmoid', kernel_initializer='lecun_normal', name='output')(x)
model = Model(inputs=[input_title_seq, input_body_seq, input_answer_seq], outputs=[output])

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title_seq (InputLayer)    [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_body_seq (InputLayer)     [(None, 30, 10)]     0                                            
__________________________________________________________________________________________________
input_answer_seq (InputLayer)   [(None, 30, 10)]     0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 300)      17402100    input_title_seq[0][0]            
______________________________________________________________________________________________

# Make predictions

In [11]:
Y_test = model.predict(X_test)

In [12]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.858963,0.488851,0.076877,0.513000,0.785699,0.696760,0.601331,0.569262,0.462323,...,0.826775,0.878116,0.587263,0.938026,0.942459,0.760103,0.161204,0.107862,0.777400,0.901673
1,46,0.863180,0.495841,0.004022,0.664495,0.796399,0.869256,0.543999,0.440929,0.138981,...,0.728317,0.939232,0.634385,0.968073,0.977812,0.856329,0.858503,0.200064,0.189376,0.897131
2,70,0.908018,0.654286,0.054639,0.791231,0.843606,0.841036,0.619430,0.571560,0.332857,...,0.834387,0.909332,0.616017,0.960270,0.963443,0.831511,0.088866,0.038384,0.749186,0.914858
3,132,0.885925,0.541984,0.000628,0.785290,0.804378,0.971710,0.549636,0.445561,0.097939,...,0.738136,0.971122,0.705463,0.984160,0.993544,0.925627,0.961298,0.129717,0.414469,0.900931
4,200,0.878523,0.507538,0.032041,0.727021,0.818258,0.835692,0.590057,0.522105,0.391802,...,0.784225,0.921059,0.624133,0.964680,0.965199,0.829786,0.262319,0.103729,0.672177,0.906887


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.876507,0.582532,0.031026,0.715009,0.802465,0.858255,0.571049,0.486559,0.225186,...,0.771773,0.927123,0.647866,0.960786,0.969049,0.853140,0.553692,0.141644,0.502964,0.896126
std,2812.670060,0.049268,0.117484,0.048631,0.101580,0.096804,0.112673,0.043356,0.070918,0.123596,...,0.086296,0.027715,0.040159,0.017285,0.016524,0.039914,0.313475,0.068115,0.257746,0.025221
min,39.000000,0.703770,0.326556,0.000147,0.380409,0.371458,0.290459,0.471601,0.326204,0.019721,...,0.509934,0.825121,0.564644,0.890698,0.898413,0.737635,0.004323,0.008894,0.046060,0.801132
25%,2572.000000,0.845694,0.481596,0.003907,0.666260,0.752656,0.826362,0.538761,0.437058,0.131862,...,0.708623,0.909934,0.619220,0.950458,0.960103,0.826083,0.254176,0.095612,0.272788,0.880887
50%,5093.000000,0.880923,0.579497,0.011258,0.729158,0.816751,0.894501,0.564502,0.467778,0.207767,...,0.772659,0.929432,0.641076,0.963283,0.970893,0.852566,0.617406,0.137067,0.485737,0.896762
75%,7482.000000,0.912906,0.674202,0.036110,0.787304,0.867701,0.932098,0.602320,0.530967,0.299156,...,0.845808,0.948412,0.670296,0.973754,0.981327,0.882735,0.835507,0.176037,0.725544,0.913223
max,9640.000000,0.977713,0.882727,0.386016,0.901998,0.988860,0.989292,0.707520,0.742754,0.755844,...,0.953996,0.983587,0.780471,0.992060,0.996821,0.950361,0.983123,0.502806,0.987506,0.962982
